In [73]:
import pandas as pd
import altair as alt

data = pd.read_csv("./Databases/sales_compiled_db.csv")
data.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,...,Genre 3,Genre 4,Genre 5,Genre 6,Total Sales,North American Sales,Japan Sales,PAL Sales,Other Sales,Console
0,2,The Legend of Zelda: Breath of the Wild,3-Mar-17,"['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4300,"['Adventure', 'RPG']",30000,2.5K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['NS', 'WiiU']"
1,3,Undertale,15-Sep-15,"['tobyfox', '8-4']",4.2,3.5K,3500,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",28000,679,...,RPG,Turn Based Strategy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['PC']
2,4,Hollow Knight,24-Feb-17,['Team Cherry'],4.4,3K,3000,"['Adventure', 'Indie', 'Platform']",21000,2.4K,...,Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Series', 'PC']"
3,5,Minecraft,18-Nov-11,['Mojang Studios'],4.3,2.3K,2300,"['Adventure', 'Simulator']",33000,1.8K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['iOS', 'And', 'Series']"
4,9,NieR: Automata,23-Feb-17,"['PlatinumGames', 'Square Enix']",4.3,2.9K,2900,"['Brawler', 'RPG']",18000,1.1K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['PS4', 'PC']"


In [74]:
from io import StringIO
import pandas as pd
import altair as alt
import re


# Define column names
columns = [
    "ID", "Title", "Release Date", "Team", "Rating", "Times Listed", "Number of Reviews",
    "Genres", "Plays", "Playing", "Backlogs", "Wishlist", "Developer", "Publisher",
    "Year Released", "Month Released", "Game Type", "Genre 1", "Genre 2", "Genre 3",
    "Genre 4", "Genre 5", "Genre 6", "Total Sales", "North American Sales", "Japan Sales",
    "PAL Sales", "Other Sales", "Console"
]

# Load data into DataFrame
df = pd.read_csv("./Databases/sales_compiled_db.csv")

# Extracting AAA and Indie games
aaa_games = df[df['Game Type'] == 'AAA']
indie_games = df[df['Game Type'] == 'Indie']

# extract consoles from the concatenated string representation
def extract_consoles(series):
    return re.findall(r"'(.*?)'", series)

# Counting games per console for AAA games
aaa_per_console = aaa_games['Console'].apply(extract_consoles).explode().value_counts().reset_index()
aaa_per_console.columns = ['Console', 'Number of AAA Games']

# Counting games per console for Indie games
indie_per_console = indie_games['Console'].apply(extract_consoles).explode().value_counts().reset_index()
indie_per_console.columns = ['Console', 'Number of Indie Games']

# Merge AAA and Indie counts
console_counts = pd.merge(aaa_per_console, indie_per_console, on='Console', how='outer').fillna(0)

# Combine counts for each console
console_counts['Total Games'] = console_counts['Number of AAA Games'] + console_counts['Number of Indie Games']

# Remove repeating consoles
console_counts = console_counts.groupby('Console').sum().reset_index()
max_count = console_counts[['Number of AAA Games', 'Number of Indie Games']].max().max()

# Plotting
bar1 = alt.Chart(console_counts).mark_bar().encode(
    x=alt.X('Console', sort='-y'),
    y=alt.Y('Number of Indie Games', scale=alt.Scale(domain=(0, max_count))),
    color=alt.value('blue'),
    tooltip=['Console', 'Number of Indie Games']
).properties(
    title='Number of Indie Games per Console'
)

bar2 = alt.Chart(console_counts).mark_bar().encode(
    x=alt.X('Console', sort='-y'),
    y=alt.Y('Number of AAA Games', scale=alt.Scale(domain=(0, max_count))),
    color=alt.value('red'),
    tooltip=['Console', 'Number of AAA Games']
).properties(
    title='Number of AAA Games per Console'
)

alt.vconcat(bar1, bar2).configure_axisX(labelAngle=45)


alt.VConcatChart(...)

In [121]:
from io import StringIO
import pandas as pd
import altair as alt
import re

# Load data into DataFrame
df = pd.read_csv("./Databases/sales_compiled_db.csv")

# Extracting AAA and Indie games
aaa_games = df[df['Game Type'] == 'AAA']
indie_games = df[df['Game Type'] == 'Indie']

# Function to extract consoles from the concatenated string representation
def extract_consoles(series):
    return re.findall(r"'(.*?)'", series)

# Counting games per console for AAA games
aaa_per_console = aaa_games['Console'].apply(extract_consoles).explode().value_counts().reset_index()
aaa_per_console.columns = ['Console', 'Number of AAA Games']

# Counting games per console for Indie games
indie_per_console = indie_games['Console'].apply(extract_consoles).explode().value_counts().reset_index()
indie_per_console.columns = ['Console', 'Number of Indie Games']

# Merge AAA and Indie counts
console_counts = pd.merge(aaa_per_console, indie_per_console, on='Console', how='outer').fillna(0)

# Compute total counts for Indie and AAA games separately
console_counts['Total Indie Games'] = console_counts['Number of Indie Games'].sum()
console_counts['Total AAA Games'] = console_counts['Number of AAA Games'].sum()

# Calculate proportions for each game type
console_counts['Indie Proportion'] = console_counts['Number of Indie Games'] / console_counts['Total Indie Games']
console_counts['AAA Proportion'] = console_counts['Number of AAA Games'] / console_counts['Total AAA Games']

# Reshape data for plotting
melted_counts = pd.melt(console_counts, id_vars='Console', value_vars=['Indie Proportion', 'AAA Proportion'], 
                        var_name='Game Type', value_name='Proportion')

# Plotting

aaa_bar = alt.Chart(console_counts).mark_bar(opacity=0.7, color='red').encode(
    x=alt.X('Console', axis=alt.Axis(title='Console'), sort=alt.SortField('AAA Proportion', order='descending')),
    y='AAA Proportion',
    tooltip=['Console', alt.Tooltip('AAA Proportion', format='.2%')],
).properties(
    title='Proportion of AAA (red) and Indie Games (blue) per Console'
)

# Plotting Indie proportions overlayed on top
indie_bar = alt.Chart(console_counts).mark_bar(opacity=0.7, color='blue').encode(
    x=alt.X('Console', axis=alt.Axis(title='Console'), sort=alt.SortField('Indie Proportion', order='descending')),
    y='Indie Proportion',
    tooltip=['Console', alt.Tooltip('Indie Proportion', format='.2%')],
)

overlayed_chart = (indie_bar + aaa_bar).resolve_scale(color='independent')

overlayed_chart.configure_axisX(labelAngle=45)


alt.LayerChart(...)